In [1]:
#Import packages
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../molecular-state-classes-and-functions/')

import numpy as np
import sympy
import pickle
import matplotlib.pyplot as plt
%matplotlib notebook
from tqdm.notebook import tqdm

#Import custom functions and classes used to manipulate molecular states and Hamiltonian
from functions import *
from classes import *

#Import basis transformation functions
from basis_transform_functions import *

#Define custom range function
def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(x)
        x += dx
    return range_list

In [3]:
#Make list of quantum numbers that defines the basis for the matrices
QN_uncoupled = make_QN(0,4,1/2,1/2)

#Get H_0 from file (H_0 should be in rad/s)
H_0_EB = make_hamiltonian("TlF_X_state_hamiltonian_J0to4.pickle")

E = np.array((0,0,0))
B = np.array((0,0,0.001))
H0_uncoupled = H_0_EB(E,B)

In [4]:
#Make list of quantum numbers for coupled basis
Jmin = 0
Jmax = 4
I_F = 1/2
I_Tl = 1/2
QN_coupled = [CoupledBasisState(F,mF,F1,J,I_F,I_Tl)
      for J  in ni_range(Jmin, Jmax+1)
      for F1 in ni_range(np.abs(J-I_F),J+I_F+1)
      for F in ni_range(np.abs(F1-I_Tl),F1+I_Tl+1)
      for mF in ni_range(-F, F+1)
     ]

In [ ]:
S_trans = generate_transform_matrix(QN_coupled,QN_uncoupled).T

In [ ]:
H0_coupled = S_trans.conj().T @ H0_uncoupled @ S_trans
# H0_coupled = S @ H0_uncoupled @ S.conj().T 

In [ ]:
#Diagonalize both Hamiltonians
D_uc, V_uc = np.linalg.eigh(H0_uncoupled)
D_c, V_c = np.linalg.eigh(H0_coupled)

print(np.allclose(D_c,D_uc))

In [ ]:
#Print some of the states
index=30
state_uc = vector_to_state(V_uc[:,index],QN_uncoupled)
state_c = vector_to_state(V_c[:,index],QN_coupled)

print("Uncoupled:\n|{}> = ".format(index))
state_uc.remove_small_components().transform_to_coupled().print_state()
print("Coupled:\n|{}> = ".format(index))
state_c.print_state()

In [ ]:
#Testing function for picking out sub-space of Hamiltonian
QN_reduced = make_QN(1,2,1/2,1/2)
H_red = reduced_basis_hamiltonian(QN_uncoupled, H0_uncoupled, QN_reduced)

In [ ]:
np.allclose(H_red, H0_uncoupled[4:36,4:36])

In [ ]:
#Save the transformation matrix to file
filename = "UC_to_C_j0to4.pickle"
with open(filename,"wb+")as f:
    pickle.dump(S_trans,f)

In [13]:
S_trans

array([[ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.70710678,  0.        ,  0.70710678, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.70710678,  0.        ,  0.70710678, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.35355339,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.35355339,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.        ]])